In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm

Using TensorFlow backend.


In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [4]:
sales = pd.read_csv("sales_train.csv")

In [5]:
test = pd.read_csv('test.csv')

In [6]:
pre_df = sales.copy()
pre_df = pre_df.pivot_table(
    index=['shop_id', 'item_id'],
    values=['item_cnt_day'],
    columns=['date_block_num'],
    fill_value=0,
    aggfunc='sum'
).reset_index()

pre_df

shop_id item_id item_cnt_day                        ...        \
date_block_num                            0   1  2  3  4  5  6  7  ... 24 25   
0                    0      30            0  31  0  0  0  0  0  0  ...  0  0   
1                    0      31            0  11  0  0  0  0  0  0  ...  0  0   
2                    0      32            6  10  0  0  0  0  0  0  ...  0  0   
3                    0      33            3   3  0  0  0  0  0  0  ...  0  0   
4                    0      35            1  14  0  0  0  0  0  0  ...  0  0   
...                ...     ...          ...  .. .. .. .. .. .. ..  ... .. ..   
424119              59   22154            1   0  0  0  0  0  0  0  ...  0  0   
424120              59   22155            0   0  0  0  0  0  1  0  ...  0  0   
424121              59   22162            0   0  0  0  0  0  0  0  ...  0  9   
424122              59   22164            0   0  0  0  0  0  0  0  ...  0  2   
424123              59   22167            0   0  0  0  0  0  0  0  ...  0  0   

                                        
date_block_num 26 27 28 29 30 31 32 33  
0               0  0  0  0  0  0  0  0  
1               0  0  0  0  0  0  0  0  
2               0  0  0  0  0  0  0  0  
3               0  0  0  0  0  0  0  0  
4               0  0  0  0  0  0  0  0  
...            .. .. .. .. .. .. .. ..  
424119          0  0  0  0  0  0  0  0  
424120          0  0  0  0  0  0  0  0  
424121          4  1  1  0  0  1  0  0  
424122          1  2  0  0  1  0  0  0  
424123          0  0  0  0  0  0  0  0  

[424124 rows x 36 columns]

In [7]:
full_train_df = test.copy()
full_train_df = full_train_df.merge(pre_df, how='left', on=['shop_id', 'item_id']).fillna(0).drop(
    ['ID', 'shop_id', 'item_id'], axis=1)
full_train_df

/home/nitin/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/home/nitin/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X_train, y_train = full_train_df.values[:,:-2], full_train_df.values[:, -2:-1].ravel()
X_valid, y_valid = full_train_df.values[:,1:-1], full_train_df.values[:, -1:].ravel()
X_test = full_train_df.values[:, 2:]

In [9]:
subsequences = 2
timesteps = X_train.shape[1]//subsequences
X_train = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))
X_valid = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))
X_test = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))

In [10]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# CNN-LSTM

In [11]:
modelCNNLSTM = Sequential()
modelCNNLSTM.add(TimeDistributed(Conv1D(filters=64, kernel_size=8, activation='relu'), input_shape=(None, X_train.shape[2], X_train.shape[3])))
modelCNNLSTM.add(TimeDistributed(MaxPooling1D(pool_size=4)))
modelCNNLSTM.add(TimeDistributed(Flatten()))
modelCNNLSTM.add(LSTM(64, activation='relu'))
modelCNNLSTM.add(Dense(1))
modelCNNLSTM.compile(optimizer='adam', loss='mse')
history = modelCNNLSTM.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))


Train on 214200 samples, validate on 214200 samples
Epoch 1/10
214200/214200 [==============================] - 25s 118us/step - loss: 2.9094 - val_loss: 23.6095
Epoch 2/10
214200/214200 [==============================] - 25s 119us/step - loss: 3.4505 - val_loss: 23.2125
Epoch 3/10
214200/214200 [==============================] - 24s 114us/step - loss: 2.7396 - val_loss: 24.8155
Epoch 4/10
214200/214200 [==============================] - 24s 114us/step - loss: 2.7408 - val_loss: 25.6653
Epoch 5/10
214200/214200 [==============================] - 25s 116us/step - loss: 3.6487 - val_loss: 24.1482
Epoch 6/10
214200/214200 [==============================] - 24s 113us/step - loss: 2.6968 - val_loss: 24.0950
Epoch 7/10
214200/214200 [==============================] - 25s 115us/step - loss: 3.5532 - val_loss: 24.3316
Epoch 8/10
214200/214200 [==============================] - 25s 116us/step - loss: 3.1146 - val_loss: 21.6845
Epoch 9/10
214200/214200 [==============================] - 25s 116u

In [12]:
CNNLSTMpred = modelCNNLSTM.predict(X_valid, verbose=0)
CNNLSTMpred = CNNLSTMpred.reshape((CNNLSTMpred.shape[0]))
CNNLSTMresults = metrics(CNNLSTMpred, y_valid)
CNNLSTMresults

{'Explained Variance Score': 0.17398698407557245,
 'Max Error': 2160.1855239868164,
 'Mean Absolute Error': 0.4449018867276296,
 'Mean Squared Error': 25.452177425799036,
 'Root Mean Squared Error': 5.045015106597703,
 'Median Absolute Error': 0.20553123950958252,
 'R² Score': 0.17398421734771163,
 'Mean Absolute Percentage Error': 157.97047133787464}